In [ ]:
import sys
sys.path.insert(0, '../../Models/LSTM')

In [1]:
# Import libraries
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from LSTM import LSTM
import pandas as pd
import hyperopt
from hyperopt import fmin, tpe, Trials, hp
import hyperopt.pyll.stochastic
from hyperopt import pyll

# Hyperopt function
def search_reg(space, max_eval, output_path):
    T = Trials()
    best = fmin(optimiser, space, algo=tpe.suggest, max_evals=max_eval, trials=T, pass_expr_memo_ctrl=True)
    
    logs = []
    results = []
    
    T.trials
    for i in T.trials:
        logs.append(i['misc']['vals'])
        results.append(i['result']['loss'])
    
    trial_data = pd.DataFrame(logs)
    trial_data['F1'] = results
    trial_data.to_csv(output_path)
    
    return hyperopt.space_eval(space, best)

In [2]:
x = pd.read_csv('../../Data/Missing_0_Mask/X_train_0_WELL.csv')
y = pd.read_csv('../../Data/Y_train.csv')
xt = pd.read_csv('../../Data/Missing_0_Mask/X_test_0_WELL.csv')
yt = pd.read_csv('../../Data/Y_test.csv')

In [3]:
def Kfold(data, labels, window_size = 1, hidden_dim = 16, n_layers = 1, optim = 'adamw', lr = 0.0001, weight_decay = 0.1, n_epoch = 100, batchsize = 1024, momentum = None, run_name = 'testrun', best = None):
    
    kf = StratifiedKFold(n_splits = 5, shuffle = True)
    kf.get_n_splits(data, labels)
    
    VS_arr = []
    
    for fold, (train_index, test_index) in enumerate(kf.split(data, labels)):
        print('--------------------------------')
        print(f'FOLD {fold}')
        print('--------------------------------')
        
        x_train, x_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]

        Model = LSTM(window_size, 26, 12, hidden_dim, n_layers)
        Model.fit( x=x_train, y=y_train, xt = x_test, yt = y_test, optim = optim, lr = lr, weight_decay = weight_decay, n_epoch = n_epoch, batchsize = batchsize, momentum = momentum, run_name = run_name)
        f1 = f1_score(yt,Model.predict(xt, yt).cpu().numpy(),average='macro')
        VS_arr.append(f1)
        
        print(f'Fold {fold} Validation Score: {f1}')
        
        # Early Stopping
        if best != None:
            if f1 < best:
                print('********************************')
                print('F1 Score too low ... stopped early.')
                print('********************************')
                return f1

    
    final_F1 = min(VS_arr)
    print('--------------------------------')
    print(f'\n\nMinimum Validation Score: {final_F1}')
    print('--------------------------------')
    
    return final_F1

In [4]:
def optimiser(expr, memo, ctrl):
    try:
        config = pyll.rec_eval(expr, memo=memo)
        print(config)
        if len(ctrl.trials.losses()) > 1:
            best_score = -min(value for value in ctrl.trials.losses() if value is not None)
        else:
            best_score = -5

        return -Kfold(x, y, window_size = config['window_size'], hidden_dim = config['hidden_dim'], n_layers = config['n_layers'], optim = 'adamw', lr = 0.0001, weight_decay = 0.1, n_epoch = 5, batchsize = 256, momentum = None, run_name = 'LSTM_search_new', best = best_score)
    
    except:
        return 99

In [5]:
space = {'window_size':hp.randint('window_size', 10),
         'hidden_dim':hp.randint('hidden_dim',500),
         'n_layers':hp.choice('n_layers',[1,2,3])}

In [6]:
search_reg(space, 50, '1_LSTM.csv')

{'hidden_dim': 399, 'n_layers': 3, 'window_size': 8}
--------------------------------                    
FOLD 0                                              
--------------------------------                    
Device is cuda:0.                                   

  0%|          | 0/50 [00:03<?, ?it/s, best loss: ?]

wandb: Currently logged in as: 2djagpal (use `wandb login --relogin` to force relogin)



Loading Data ...                                    

Starting training...                                

Training:                                           
Epoch: 1 ... Accuracy: 0.6209451435698968 ... Score: -1.1412694840283295

Validation:                                         
Epoch: 1 ... Accuracy: 0.6199279804188754 ... Score: -1.1451898309718371

Training:                                           
Epoch: 2 ... Accuracy: 0.5573211676961325 ... Score: -1.153152258417271

Validation:                                         
Epoch: 2 ... Accuracy: 0.549740926002657 ... Score: -1.1781566233666378

Training:                                           
Epoch: 3 ... Accuracy: 0.6620671758464259 ... Score: -0.8759111946929117

Validation:                                         
Epoch: 3 ... Accuracy: 0.6418456833103378 ... Score: -0.9345635254567434

Training:                                           
Epoch: 4 ... Accuracy: 0.6860289531913439 ... Score: -0.829305575133916

Valida

train_accuracy,0.67937
train_batch_loss,1.11437
train_epoch_loss,1.53426
train_score,-0.86316
valid_accuracy,0.66347
valid_epoch_loss,1.60934
valid_score,-0.91164


Loading Data ...                                    

Starting training...                                

Training:                                           
Epoch: 1 ... Accuracy: 0.58665711243698 ... Score: -1.2050817271085605

Validation:                                         
Epoch: 1 ... Accuracy: 0.5739079546522456 ... Score: -1.245885874533323

Training:                                           
Epoch: 2 ... Accuracy: 0.6368125466542932 ... Score: -1.0100563695991815

Validation:                                         
Epoch: 2 ... Accuracy: 0.6279655876498279 ... Score: -1.0401288754474545

Training:                                           
Epoch: 3 ... Accuracy: 0.6660102583379698 ... Score: -0.9425675639597655

Validation:                                         
Epoch: 3 ... Accuracy: 0.6508786768160887 ... Score: -0.9924423969039137

Training:                                           
Epoch: 4 ... Accuracy: 0.4561692593727741 ... Score: -1.3286599391932372

Valida

...

train_accuracy,0.71858
train_batch_loss,3.62973
train_epoch_loss,1.31624
train_score,-0.73438
valid_accuracy,0.70308
valid_epoch_loss,1.4306
valid_score,-0.78276


Loading Data ...                                                                  

Starting training...                                                              

Training:                                                                         
Epoch: 1 ... Accuracy: 0.5018015651297297 ... Score: -1.518196795627547           

Validation:                                                                       
Epoch: 1 ... Accuracy: 0.4943123326057334 ... Score: -1.5443389234653122          

Training:                                                                         
Epoch: 2 ... Accuracy: 0.6543622010918317 ... Score: -0.9653472364610298          

Validation:                                                                       
Epoch: 2 ... Accuracy: 0.6377790972349777 ... Score: -1.0123748948112583          

Training:                                                                         
Epoch: 3 ... Accuracy: 0.5431830996745008 ... Score: -1.191760829681079          

train_accuracy,0.71218
train_batch_loss,1.50358
train_epoch_loss,1.58317
train_score,-0.77135
valid_accuracy,0.69365
valid_epoch_loss,1.68856
valid_score,-0.82994


Loading Data ...                                                                  

Starting training...                                                              

Training:                                                                         
Epoch: 1 ... Accuracy: 0.6275341517799934 ... Score: -1.0496666517159188          

Validation:                                                                       
Epoch: 1 ... Accuracy: 0.6176298466914136 ... Score: -1.0850539078952426          

Training:                                                                         
Epoch: 2 ... Accuracy: 0.6547028645633101 ... Score: -0.9469536783111635          

Validation:                                                                       
Epoch: 2 ... Accuracy: 0.6385522611841796 ... Score: -0.9977419127478075          

Training:                                                                         
Epoch: 3 ... Accuracy: 0.6885855310932841 ... Score: -0.8079082515313838         

{'hidden_dim': 474, 'n_layers': 1, 'window_size': 7}